# 第7章
モデルのアンサンブル手法を学ぶ。

---
ソースコードは以下から引用しています: https://github.com/ghmagazine/kagglebook/tree/master/ch07

ライセンス: https://github.com/ghmagazine/kagglebook/blob/master/LICENSE

## スタッキング

### データ準備

In [1]:
import numpy as np
import pandas as pd

# train_xは学習データ、train_yは目的変数、test_xはテストデータ
# pandasのDataFrame, Seriesで保持します。（numpyのarrayで保持することもあります）

train = pd.read_csv('data/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x = pd.read_csv('data/sample-data/test_preprocessed.csv')

# neural net用のデータ
train_nn = pd.read_csv('data/sample-data/train_preprocessed_onehot.csv')
train_x_nn = train_nn.drop(['target'], axis=1)
train_y_nn = train_nn['target']
test_x_nn = pd.read_csv('data/sample-data/test_preprocessed_onehot.csv')

### モデルの準備

In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# tensorflowの警告抑制
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


# xgboostによるモデル
class Model1Xgb:

    def __init__(self):
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71,
                  'eval_metric': 'logloss'}
        num_round = 10
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        dvalid = xgb.DMatrix(va_x, label=va_y)
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        self.model = xgb.train(params, dtrain, num_round, evals=watchlist)

    def predict(self, x):
        data = xgb.DMatrix(x)
        pred = self.model.predict(data)
        return pred


# ニューラルネットによるモデル
class Model1NN:

    def __init__(self):
        self.model = None
        self.scaler = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        self.scaler = StandardScaler()
        self.scaler.fit(tr_x)

        batch_size = 128
        epochs = 5

        tr_x = self.scaler.transform(tr_x)
        va_x = self.scaler.transform(va_x)
        model = Sequential()
        model.add(Dense(256, activation='relu', input_shape=(tr_x.shape[1],)))
        model.add(Dropout(0.2))
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(1, activation='sigmoid'))

        model.compile(loss='binary_crossentropy', optimizer='adam')

        history = model.fit(tr_x, tr_y,
                            batch_size=batch_size, epochs=epochs,
                            verbose=1, validation_data=(va_x, va_y))
        self.model = model

    def predict(self, x):
        x = self.scaler.transform(x)
        pred = self.model.predict_proba(x).reshape(-1)
        return pred


# 線形モデル
class Model2Linear:

    def __init__(self):
        self.model = None
        self.scaler = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        self.scaler = StandardScaler()
        self.scaler.fit(tr_x)
        tr_x = self.scaler.transform(tr_x)
        self.model = LogisticRegression(solver='lbfgs', C=1.0)
        self.model.fit(tr_x, tr_y)

    def predict(self, x):
        x = self.scaler.transform(x)
        pred = self.model.predict_proba(x)[:, 1]

        return pred

Using TensorFlow backend.


### スタッキング

In [3]:
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(model, train_x, train_y, test_x):
    preds_val = []  # 各foldにおける(valid以外の)trainで学習したモデルのvalidに対する予測を格納
    preds_test = []  # 各foldにおけるtestに対する予測を格納
    va_idxes = []  # 各foldにおいてvaldとして割り当てられたデータのインデックスを格納

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)  # trainで学習, valは学習経過のバリデーションに使用
        pred_val = model.predict(va_x)  # このfoldにおいて(valid以外の)trainで学習したモデルを使ってvalidに対して予測
        preds_val.append(pred_val)
        pred_test = model.predict(test_x)  # このfoldにおいて(valid以外の)trainで学習したモデルを使ってtest（foldに関わらず同じデータ）に対して予測
        preds_test.append(pred_test)
        va_idxes.append(va_idx)
    # - - - クロスバリデーション終わり - - -

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds_val = np.concatenate(preds_val, axis=0)
    # どうしてインデックスを昇順にソートし直す必要がある？
    # → preds_valの並びはKFold.split()によってランダムになっており、
    #     このままだとモデル評価（log_loss()の箇所）時に、特徴量preds_valとラベルtrain_yが正しく対応しないから
    #     （preds_valをインデックス昇順にソートすることで、train_yの並びと合うようになる）
    order = np.argsort(va_idxes)  # va_idxesを昇順ソートした後のインデックス
    pred_train = preds_val[order]
    
    # テストデータに対する予測値の平均をとる
    # （平均ではなく、学習データ全体に対して学習し直したモデルで予測する方法もある（P.366参照））
    preds_test_mean = np.mean(preds_test, axis=0)
    
    return pred_train, preds_test_mean

#### 1層目：オリジナルの学習データで学習したモデル

In [4]:
# 1層目のモデル
# pred_train_1a, pred_train_1bは、各foldのモデルのバリデーションデータに対する予測値
# pred_test_1a, pred_test_1bは、各foldのモデルのテストデータに対する予測値の平均
# （pred_train_xx, pred_test_xxはいずれも、予測対象のレコードの目的変数を見ていない状況で学習したモデルによる予測値）
model_1a = Model1Xgb()
pred_train_1a, pred_test_1a = predict_cv(model_1a, train_x, train_y, test_x)

model_1b = Model1NN()
pred_train_1b, pred_test_1b = predict_cv(model_1b, train_x_nn, train_y, test_x_nn)

# 1層目のモデルの評価
print(f'logloss: {log_loss(train_y, pred_train_1a, eps=1e-7):.4f}')
print(f'logloss: {log_loss(train_y, pred_train_1b, eps=1e-7):.4f}')

[0]	train-logloss:0.540879	eval-logloss:0.550034
[1]	train-logloss:0.452692	eval-logloss:0.47182
[2]	train-logloss:0.394818	eval-logloss:0.42026
[3]	train-logloss:0.351976	eval-logloss:0.385203
[4]	train-logloss:0.320213	eval-logloss:0.361498
[5]	train-logloss:0.296733	eval-logloss:0.344634
[6]	train-logloss:0.276105	eval-logloss:0.329003
[7]	train-logloss:0.258858	eval-logloss:0.316697
[8]	train-logloss:0.243628	eval-logloss:0.30775
[9]	train-logloss:0.231527	eval-logloss:0.300925
[0]	train-logloss:0.538915	eval-logloss:0.548639
[1]	train-logloss:0.452188	eval-logloss:0.471485
[2]	train-logloss:0.395742	eval-logloss:0.419976
[3]	train-logloss:0.354763	eval-logloss:0.384132
[4]	train-logloss:0.322183	eval-logloss:0.356264
[5]	train-logloss:0.299451	eval-logloss:0.339098
[6]	train-logloss:0.277833	eval-logloss:0.325516
[7]	train-logloss:0.263263	eval-logloss:0.315733
[8]	train-logloss:0.247804	eval-logloss:0.30592
[9]	train-logloss:0.233693	eval-logloss:0.295955
[0]	train-logloss:0.5433

In [5]:
# 予測値を特徴量としてデータフレームを作成
train_x_2 = pd.DataFrame({'pred_1a': pred_train_1a, 'pred_1b': pred_train_1b})
test_x_2 = pd.DataFrame({'pred_1a': pred_test_1a, 'pred_1b': pred_test_1b})

In [6]:
train_x_2.head()

,pred_1a,pred_1b
0,0.052786,0.145576
1,0.189076,0.077340
2,0.431136,0.339443
3,0.042489,0.020771
4,0.256559,0.679355


In [7]:
test_x_2.head()

,pred_1a,pred_1b
0,0.299180,0.577786
1,0.155138,0.049900
2,0.075771,0.032241
3,0.045092,0.004192
4,0.047573,0.012028


#### 2層目：「1層目のモデルでの予測値」という特徴量を用いて学習したモデル

In [8]:
# 2層目のモデル
model_2 = Model2Linear()
pred_train_2, pred_test_2 = predict_cv(model_2, train_x_2, train_y, test_x_2)
print(f'logloss: {log_loss(train_y, pred_train_2, eps=1e-7):.4f}')

logloss: 0.2740
